In [1]:
from dotenv import load_dotenv
import os
load_dotenv()

True

### Quick Started

In [2]:
from langchain.chat_models import init_chat_model

llm = init_chat_model(model="gpt-4o-mini", temperature=0)

In [3]:
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field

tagging_prompt = ChatPromptTemplate.from_template(
       """
Extract the desired information from the following passage.

Only extract the properties mentioned in the 'Classification' function.

Passage:
{input}
"""
)

class Classification(BaseModel):
   
   sentiment: str = Field(description="The sentiment of the text")
   aggressiveness: str = Field(description="How aggressive the text is on a scale of 1 to 10")
   language: str = Field(description="The language of the text")
   


structured_llm = llm.with_structured_output(Classification)

In [4]:
inp = "Estoy increiblemente contento de haberte conocido! Creo que seremos muy buenos amigos!"
prompt = tagging_prompt.invoke(input=inp)

response = structured_llm.invoke(prompt)

In [5]:
response

Classification(sentiment='positive', aggressiveness='1', language='Spanish')

### Finer control

In [6]:
class Classification(BaseModel):
   
   sentiment: str = Field(..., enum=["happy", "neutral", "sad"])
   aggressiveness: int = Field(
      ..., 
      description="describes how aggressive the statement is, the higher the number the more aggressive",
      enum=[1, 2, 3, 4, 5],
   )
   
   language: str = Field(
      ..., enum=["spanish", "english", "french", "german", "italian"]

   )

In [9]:
from langchain.chat_models import ChatOpenAI

tagging_prompt = ChatPromptTemplate.from_template(
    """
Extract the desired information from the following passage.

Only extract the properties mentioned in the 'Classification' function.

Passage:
{input}
"""
)

## llm = ChatOpenAI(temperature=0, model="gpt-4o-mini").with_structured_output(Classification)

In [8]:
from pydantic import BaseModel, Field
from enum import Enum
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import PydanticOutputParser

# Enums
class Sentiment(str, Enum):
    happy = "happy"
    neutral = "neutral"
    sad = "sad"

class AggressivenessLevel(int, Enum):
    one = 1
    two = 2
    three = 3
    four = 4
    five = 5

class Language(str, Enum):
    spanish = "spanish"
    english = "english"
    french = "french"
    german = "german"
    italian = "italian"

# Pydantic schema
class Classification(BaseModel):
    sentiment: Sentiment
    aggressiveness: AggressivenessLevel
    language: Language

# Parser
parser = PydanticOutputParser(pydantic_object=Classification)

# Prompt with formatting instructions
prompt = ChatPromptTemplate.from_template("""
Extract the following properties from the passage below:
- sentiment: one of {sentiments}
- aggressiveness: 1 to 5
- language: one of {languages}

Format your answer as JSON:
{format_instructions}

Passage:
{input}
""")

# LLM
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Chain
chain = prompt | llm | parser

# Run
result = chain.invoke({
    "input": "He screamed in Spanish and slammed the door.",
    "sentiments": list(Sentiment),
    "languages": list(Language),
    "format_instructions": parser.get_format_instructions()
})

print(result)


sentiment=<Sentiment.sad: 'sad'> aggressiveness=<AggressivenessLevel.four: 4> language=<Language.spanish: 'spanish'>
